In [2]:
from yolox.exp import Exp, get_exp
from yolox.data import DataPrefetcher

import cv2
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt

exp = get_exp("./exps/yolox_s_dtld_aug_960.py", "test")
train_loader = exp.get_data_loader(
            batch_size=1,
            is_distributed=False,
            no_aug=False,
            cache_img=False,
        )
prefetcher = DataPrefetcher(train_loader)

def draw_data(fetcher, exp, save_path=None, show=False):
    inps, targets = fetcher.next()
    inps, targets = exp.preprocess(inps, targets, exp.input_size)
    img1 = np.uint8(inps[0].permute(1,2,0).cpu().numpy())
    img1 = img1.astype(np.uint8).copy()

    for ttt in targets[0]:
        if ttt[4] > 0:
            lt = (int(ttt[1] - ttt[3]/2.0), int(ttt[2] - ttt[4]/2.0))
            rb = (int(ttt[1] + ttt[3]/2.0), int(ttt[2] + ttt[4]/2.0))
            img1 = cv2.rectangle(img1, lt, rb, (0,255,0),1)
            if save_path:
                cv2.imwrite(save_path, img1)
    if show:
        plt.figure(dpi=100, figsize=(8,8))
        plt.imshow(img1[:,:,[2,1,0]])

In [3]:
for i in tqdm(range(50)):
    draw_data(prefetcher, exp, "draw/%d.jpg"%i)

100%|██████████| 50/50 [00:13<00:00,  3.72it/s]


In [1]:
!rm -r draw/*